In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import tqdm

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

LOAD_DATA = True
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 100

cpu


In [24]:
class DataLoader():
    def __init__(self, batch_size, path):
        self.batch_size = batch_size
        self.path = path
        self.current_batch = 0
        # load data
        input1_pd = pd.read_excel(open(self.path, 'rb'), sheet_name = 'Sheet1', header = None)
        input_ten = torch.from_numpy( np.array(input1_pd) ).float()
        self.input = input_ten.reshape(-1,1,64,64)
        
        label = pd.read_excel(open(self.path, 'rb'), sheet_name = 'Sheet2', header = None)
        self.label = torch.from_numpy( np.array(label) ).float()
    
        self.data_size = len(label)
        print(self.data_size)
    
    def getbatchnum(self):
        return math.ceil(self.data_size / self.batch_size)
    
    def getbatch(self):
        current_batch = self.current_batch
        length = self.batch_size

        if current_batch + length <= self.data_size :
            data_return = self.input[ current_batch : current_batch + length ]
            self.current_batch += length
            return data_return, self.label[ current_batch : current_batch + length ]
        else:
            data_return = self.input[ current_batch : -1 ]
            self.current_batch = 0
            return data_return, self.label[ current_batch : -1]
        
    def reset(self): #어디까지 리턴했는지 초기화
        self.current_batch = 0
        

In [29]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 24, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #32 x 32 x 24
            
            nn.Conv2d(24, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #16 x 16 x 32
            
            nn.Conv2d(32, 40, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(40, 40, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #8 x 8 x 40
            
            nn.Conv2d(40, 48, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #4 x 4 x 48
            
            nn.Conv2d(48, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
            #2 x 2 x 64
        )
        self.fcc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 48),
            nn.Dropout(0.4),
            nn.Linear(48, 5)
        )
    def forward(self, x):
        output = self.cnn(x)
        output = output.view(output.size(0), -1)
        output = self.fcc(output)
        return output

In [26]:
if LOAD_DATA:
    trainpath = 'traindata.xlsx'
    valpath = 'valdata.xlsx'
    testpath = 'testdata.xlsx'
    
    traindata = DataLoader(batch_size = BATCH_SIZE, path = trainpath)
    valdata = DataLoader(batch_size = BATCH_SIZE, path = valpath)
    testdata = DataLoader(batch_size = 1, path = testpath)

100
10


In [30]:
net = Net()
optimizer = optim.Adam(net.parameters(), LEARNING_RATE, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
loss_function = nn.MSELoss()

In [32]:
cost_list = []
for epoch in range(EPOCHS):
        cumcost = 0
        total = 0
        for idx in range(traindata.getbatchnum()):
            x, y = traindata.getbatch()
            x = x.view(-1, 1, 64, 64)
            
            net.zero_grad()
            pred = net(x)
            loss = loss_function(pred, y)
            loss.backward()
            optimizer.step()
            
            cumcost += loss
            total += 1
        cost_list.append(cumcost / total)
        print("epochs : ", epoch, "loss: ", cumcost / total)
        

epochs :  0 loss:  tensor(74.6175, grad_fn=<DivBackward0>)
epochs :  1 loss:  tensor(33.0421, grad_fn=<DivBackward0>)
epochs :  2 loss:  tensor(29.8571, grad_fn=<DivBackward0>)
epochs :  3 loss:  tensor(21.3580, grad_fn=<DivBackward0>)
epochs :  4 loss:  tensor(26.2496, grad_fn=<DivBackward0>)
epochs :  5 loss:  tensor(21.6995, grad_fn=<DivBackward0>)
epochs :  6 loss:  tensor(25.6292, grad_fn=<DivBackward0>)
epochs :  7 loss:  tensor(19.5356, grad_fn=<DivBackward0>)
epochs :  8 loss:  tensor(17.3498, grad_fn=<DivBackward0>)
epochs :  9 loss:  tensor(17.9261, grad_fn=<DivBackward0>)
epochs :  10 loss:  tensor(17.8681, grad_fn=<DivBackward0>)
epochs :  11 loss:  tensor(17.3151, grad_fn=<DivBackward0>)
epochs :  12 loss:  tensor(17.5210, grad_fn=<DivBackward0>)
epochs :  13 loss:  tensor(14.4861, grad_fn=<DivBackward0>)
epochs :  14 loss:  tensor(16.1159, grad_fn=<DivBackward0>)
epochs :  15 loss:  tensor(17.6188, grad_fn=<DivBackward0>)
epochs :  16 loss:  tensor(16.2405, grad_fn=<DivBa

In [39]:
a_error = []
b_error = []
x_error = []
y_error = []
d_error = []
predictions = []
with torch.no_grad():
    for idx in range(testdata.getbatchnum()):
        data, label = testdata.getbatch()
        data = data.view(-1, 1, 64, 64)
        pred = net(data)
        print(label)
        
        error = (label - pred) / label
        print(error)
        predictions.append(pred)
        a_error.append(error[0][0])
        b_error.append(error[0][1])
        x_error.append(error[0][2])
        y_error.append(error[0][3])
        d_error.append(error[0][4])



tensor([[13.9004,  8.3020,  3.1929,  0.9460,  5.3529]])
tensor([[-0.3174, -0.8380, -1.6982, -0.2449, -1.0533]])
tensor([[ 7.2843, 11.5200,  0.4819,  3.1730,  4.9455]])
tensor([[ -0.8388,  -0.0437, -12.5304,   0.2424,  -0.0867]])
tensor([[13.8007,  9.4433, 10.4321,  5.6971, 11.9657]])
tensor([[-0.0584, -0.3589,  0.3969, -0.3301,  0.0176]])
tensor([[ 6.1393, 14.7856,  5.2098,  0.7488,  7.5268]])
tensor([[-1.0322,  0.1609, -0.1838, -1.9622, -0.3712]])
tensor([[ 8.2565, 11.3021,  1.9015,  6.5539,  9.4442]])
tensor([[-0.5499, -0.2553, -1.6463,  0.7185, -0.3646]])
tensor([[10.9988,  9.4843,  0.3896,  4.8732,  6.7082]])
tensor([[ -0.2882,  -0.3568, -13.4208,   0.2287,  -0.7635]])
tensor([[ 6.0805,  9.5988,  2.7416,  5.2903, 12.2757]])
tensor([[-0.4125,  0.0444, -0.7418,  0.3853, -0.0841]])
tensor([[12.0085, 13.7224,  0.6267,  3.0145,  7.4350]])
tensor([[ 0.0413,  0.1830, -8.8398,  1.1398,  0.0216]])
tensor([[14.5853, 12.9005,  6.5907,  4.3014,  1.8273]])
tensor([[-0.1664, -0.0583,  0.2636, -0